# Test 2 implementacion de modelo

Se realiza el primer ejercicio de como poder implementar el modelo de regresion lineal a partir de los datos que se tienen.

In [3]:
# Importar las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [4]:
# Cargar el archivo con el encabezado correcto (fila 5, índice 4)
file_path = "../data/raw/archivo 3/Reporte Cronicos ESE Oriente ABRIL 2023.parquet"
try:
    df = pd.read_parquet(file_path)
except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no se encontró. Asegúrate de que esté en la misma carpeta.")
    exit()



In [5]:
df_select = df[[
    'edad_(años_cumplidos)',
    '8_sexo',
    '20_dxdm',
    '23_peso',
    '24_talla',
    '25_tenartsis',
    '26_tenartditlica',
    '31_colesteroltotal',
    '32_hdl',
    'cigarrillo',
    '36_rcbeieca',
    '37_rcbeara2',
 ]]
df_select

,edad_(años_cumplidos),8_sexo,20_dxdm,23_peso,24_talla,25_tenartsis,26_tenartditlica,31_colesteroltotal,32_hdl,cigarrillo,36_rcbeieca,37_rcbeara2
0,82,M,NO,75.0,167.0,130,89,<NA>,26.0,<NA>,NO,NO
1,55,M,NO,95.0,165.0,130,70,<NA>,<NA>,No,NO,NO
2,66,F,NO,64.0,161.0,160,80,<NA>,<NA>,No,NO,NO
3,78,F,SI,64.0,160.0,160,80,<NA>,<NA>,No,NO,NO
4,55,F,SI,82.0,160.0,110,80,209.0,37.0,No,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
43186,31,F,SI,62.0,164.0,110,80,<NA>,<NA>,No,NO,NO
43187,69,F,SI,67.0,152.0,110,70,197.94,55.4,No,NO,NO
43188,55,M,SI,58.0,157.0,110,70,249.01,<NA>,No,NO,NO
43189,67,M,SI,66.0,172.0,110,70,159.05,40.77,<NA>,NO,NO


In [6]:
dic_type = {
    'edad_(años_cumplidos)': 'Int64',
    '8_sexo': 'category',
    '20_dxdm': 'category',
    '23_peso': 'Float32',
    '24_talla': 'Float32',
    '25_tenartsis': 'Float32',
    '26_tenartditlica': 'Float32',
    '31_colesteroltotal': 'Float32',
    '32_hdl': 'Float32',
    'cigarrillo': 'category',
    '36_rcbeieca': 'category',
    '37_rcbeara2': 'category',
}
df_select = df_select.astype(dic_type)

In [7]:
df_select = df_select.dropna()
df_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38822 entries, 4 to 43187
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   edad_(años_cumplidos)  38822 non-null  Int64   
 1   8_sexo                 38822 non-null  category
 2   20_dxdm                38822 non-null  category
 3   23_peso                38822 non-null  Float32 
 4   24_talla               38822 non-null  Float32 
 5   25_tenartsis           38822 non-null  Float32 
 6   26_tenartditlica       38822 non-null  Float32 
 7   31_colesteroltotal     38822 non-null  Float32 
 8   32_hdl                 38822 non-null  Float32 
 9   cigarrillo             38822 non-null  category
 10  36_rcbeieca            38822 non-null  category
 11  37_rcbeara2            38822 non-null  category
dtypes: Float32(6), Int64(1), category(5)
memory usage: 1.9 MB


In [9]:
# Convertir datos a tipos adecuados, si o no a 1 y 0
df_select['cigarrillo'] = df_select['cigarrillo'].cat.codes
df_select['20_dxdm'] = df_select['20_dxdm'].cat.codes
df_select['36_rcbeieca'] = df_select['36_rcbeieca'].cat.codes
df_select['37_rcbeara2'] = df_select['37_rcbeara2'].cat.codes
df_select['8_sexo'] = df_select['8_sexo'].cat.codes

In [11]:
diccionario_columna = {
    'edad_(años_cumplidos)':'edad',	
    '8_sexo':'sexo',
    '20_dxdm':'diabetes_mellitus',
    '23_peso':'peso',	
    '24_talla':'talla',
    '25_tenartsis':'tension_sistolica',
    '26_tenartditlica':'tension_diastolica',
    '31_colesteroltotal':'colesterol_total',	
    '32_hdl':'colesterol_hdl',
    'cigarrillo':'cigarrillo',
    '36_rcbeieca':'ieca',
    '37_rcbeara2':'ieca_2',	
}

In [12]:
df_select = df_select.rename(columns=diccionario_columna)

In [13]:
# Define las variables X (independiente) y Y (dependiente).
X = df_select[['tension_sistolica']]  # Usamos doble corchete para que sea un DataFrame
y = df_select['tension_diastolica']

# Divide los datos en conjuntos de entrenamiento (80%) y prueba (20%).
# Esto es para que el modelo aprenda de una parte de los datos y sea evaluado
# con datos que no ha visto.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea y entrena el modelo de regresión lineal.
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [15]:
# Realiza predicciones en el conjunto de datos de prueba.
y_pred = model.predict(X_test)

# Calcula las métricas de evaluación del modelo.
# El Error Cuadrático Medio (MSE) mide el error promedio de las predicciones.
mse = mean_squared_error(y_test, y_pred)
# El Coeficiente de Determinación (R^2) mide qué tan bien se ajusta el modelo.
r2 = r2_score(y_test, y_pred)

print("--- Evaluación del Modelo de Regresión Lineal ---")
print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Coeficiente de Determinación (R^2): {r2:.2f}")

# Un valor de R^2 de 0.42 indica que el 42% de la variabilidad en la
# presión diastólica puede ser explicada por la presión sistólica.

--- Evaluación del Modelo de Regresión Lineal ---
Error Cuadrático Medio (MSE): 48.12
Coeficiente de Determinación (R^2): 0.41


Coeficiente de Determinación (R 
2
 )
Valor: 0.41

Concepto Médico:
El coeficiente R 
2
  te indica qué tan bien tu variable independiente (tensión sistólica) explica la variabilidad de la variable dependiente (tensión diastólica).

Un R 
2
  de 0.41 significa que el 41% de la variabilidad en la tensión arterial diastólica de los pacientes puede ser explicada por los cambios en su tensión arterial sistólica. En otras palabras, la presión sistólica es un predictor moderadamente bueno de la diastólica, pero no es el único factor determinante. El 59% restante de la variación se debe a otras variables que no están en el modelo, como la edad del paciente, su peso, nivel de estrés, historial genético o la presencia de otras comorbilidades.

Este resultado confirma la fuerte relación que ya habías encontrado en tu análisis bivariado, pero también te enseña que, para un diagnóstico completo, un médico no puede basarse únicamente en la tensión sistólica.

Error Cuadrático Medio (MSE)
Valor: 48.12

Concepto Médico:
El MSE es una medida del error promedio de las predicciones de tu modelo. Para entenderlo en términos médicos, es útil calcular la raíz cuadrada del MSE, lo que nos da la raíz del error cuadrático medio (RMSE), que está en las mismas unidades que la variable que estamos prediciendo (mmHg).

RMSE =  
48.12

​
 ≈6.94 mmHg

El RMSE de 6.94 mmHg significa que, en promedio, las predicciones que tu modelo hace para la presión arterial diastólica de un paciente se desvían de los valores reales en casi 7 mmHg.

Interpretación en la Práctica Clínica:
Un error de ±7 mmHg puede ser clínicamente significativo, ya que un valor de 80 mmHg se considera normal, mientras que 89 mmHg puede indicar prehipertensión. Por lo tanto, aunque el modelo captura la tendencia general, no es lo suficientemente preciso como para ser utilizado para tomar decisiones críticas sobre el tratamiento de un paciente. Te sirve como una herramienta para entender la relación, pero no reemplaza una medición precisa ni la valoración médica completa.

En resumen, el modelo es útil para fines de análisis y estudio, ya que muestra una relación real y significativa entre las dos variables, pero en la práctica médica real, se requeriría un modelo mucho más preciso que incorpore otros factores relevantes del paciente para reducir ese margen de error.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test['tension_sistolica'], y=y_test, alpha=0.6, label='Datos Reales de Prueba')
plt.plot(X_test, y_pred, color='red', linewidth=2, label='Línea de Regresión')
plt.title('Modelo de Regresión Lineal: Predicción de Tensión Diastólica')
plt.xlabel('Tensión Arterial Sistólica (mmHg)')
plt.ylabel('Tensión Arterial Diastólica (mmHg)')
plt.legend()
plt.show()
